In [1]:
import pandas as pd
import os
from modules.backtest import single_crypto_backtest

In [2]:
train_start, train_end = '2021-01-02 00:00:00', '2021-09-30 00:00:00',
test_start, test_end = '2021-10-01 00:00:00', '2021-12-29 00:00:00'

file_names = ['predictions_15mins', 'predictions_30mins', 'predictions_1hour','predictions_1day', 'predictions_30days']

In [3]:
intervals = [15, 30, 60, 60*24, 60*24*30]
count = 0
backtest_result_folder = 'backtest_result'
for file_name in file_names:
    interval = file_name.split('_')[1]
    df = pd.read_csv(f'model_predictions/{file_name}.csv')
    df['signal'] =  df['prediction'].apply(lambda x: 1 if x > 0 else 0)
    df = df[['datetime','close', 'signal']]
    df['datetime'] = pd.to_datetime(df['datetime'])  # Convert 'datetime' column to datetime format
    df.set_index('datetime', inplace=True)  # Set 'datetime' as the index

    test_df = df.loc[test_start:test_end]
    final_portfolio_value, portfolio_df = single_crypto_backtest(test_df, rebalance_interval_minutes = intervals[0])
    print(f"final_portfolio_value at {interval} : {final_portfolio_value}", )

    # Check if the folder exists, and create it if it doesn't
    if not os.path.exists(backtest_result_folder):
        os.makedirs(backtest_result_folder)

    backtest_file_path = os.path.join(backtest_result_folder, f'backtest_{interval}.csv')
    portfolio_df.to_csv(backtest_file_path, index=False)

    count += 1

final_portfolio_value at 15mins : 3769.919999998222
final_portfolio_value at 30mins : 2125.5200000000477
final_portfolio_value at 1hour : 2694.640000000215
final_portfolio_value at 1day : 15402.500000000507
final_portfolio_value at 30days : 343281.6200000002
